### 코드 이해용

In [ ]:
import pandas as pd
from collections import defaultdict
from itertools import combinations

orders = pd.read_csv('./data/Sample_InputData.csv')
od_matrix = pd.read_csv('./data/Sample_OD_Matrix.csv', index_col= 0)
od_matrix.head()

sku_freq = orders.groupby('SKU_CD')['NUM_PCS'].sum()
sku_freq

# sku 빈도 높은 순서대로 리스트
skus_freq_sort = sku_freq.sort_values(ascending = False).index.tolist()
skus_freq_sort

cooc = defaultdict(int)
for _, group in orders.groupby('ORD_NO') :
  for a, b in combinations(group['SKU_CD'], 2) : # 주문별 sku 2개씩 조합
    cooc[(a,b)] += 1
    cooc[(b,a)] += 1

cooc # SKU 조합별로 N번 등장했는지 확인 가능
# ('SKU_0267', 'SKU_0246'): 1,

# candidates = [sku for sku in skus_freq_sort]
# candidates.sort(key= lambda x : cooc.get(('SKU_0117', x), 0), reverse= True)
# candidates

rack_locations = od_matrix.index[2:]
start_location = od_matrix.index[0]
dist_start = od_matrix.loc[start_location, rack_locations]
rack_sorted = dist_start.sort_values().index.tolist() # 입고 지점과 가까운 순서대로의 랙
rack_sorted

# 그리디 클러스터링
assigned = set()
clusters = []

rack_capa = 2

# 빈도 높은 순으로 배치
# 클러스터는 연관도 가장 높은 2개씩 묶음

for sku in skus_freq_sort :
  # 할당되었으면 넘어가기
  if sku in assigned :
    continue
  cluster = {sku}
  # 할당되지 않은 후보 sku들
  candidates = [s for s in skus_freq_sort if s not in assigned]
  # 특정 sku와 많이 등장한 sku부터 아예 등장하지 않은 sku까지 차례대로
  candidates.sort(key= lambda x : cooc.get((sku, x), 0), reverse= True)
  for c in candidates : # 연관성 높은 순대로
    if len(cluster) < rack_capa :
      cluster.add(c)
    else :
      break
  assigned |= cluster 
  clusters.append(cluster)

clusters

# 
sku_to_location = {}
for rack, cluster in zip(rack_sorted, clusters) :
  for sku in cluster :
    sku_to_location[sku] = rack

remaining = [s for in skus_freq_sort if s not in sku_to_location]
print(len(remaining))
idx = len(clusters)
for sku in remaining :
  rack = rack_sorted[idx // rack_capa]
  sku_to_location[sku] = rack
  idx += 1
orders['LOC'] = self.orders['SKU_CD'].map(sku_to_location)  

['WP_0008',
 'WP_0007',
 'WP_0022',
 'WP_0021',
 'WP_0009',
 'WP_0023',
 'WP_0006',
 'WP_0020',
 'WP_0036',
 'WP_0035',
 'WP_0050',
 'WP_0049',
 'WP_0010',
 'WP_0024',
 'WP_0005',
 'WP_0019',
 'WP_0037',
 'WP_0034',
 'WP_0051',
 'WP_0048',
 'WP_0064',
 'WP_0063',
 'WP_0078',
 'WP_0077',
 'WP_0011',
 'WP_0025',
 'WP_0004',
 'WP_0018',
 'WP_0038',
 'WP_0052',
 'WP_0033',
 'WP_0047',
 'WP_0065',
 'WP_0062',
 'WP_0079',
 'WP_0076',
 'WP_0012',
 'WP_0026',
 'WP_0003',
 'WP_0092',
 'WP_0017',
 'WP_0091',
 'WP_0039',
 'WP_0106',
 'WP_0053',
 'WP_0105',
 'WP_0032',
 'WP_0046',
 'WP_0066',
 'WP_0061',
 'WP_0080',
 'WP_0075',
 'WP_0013',
 'WP_0027',
 'WP_0002',
 'WP_0093',
 'WP_0016',
 'WP_0090',
 'WP_0107',
 'WP_0104',
 'WP_0040',
 'WP_0054',
 'WP_0031',
 'WP_0045',
 'WP_0120',
 'WP_0119',
 'WP_0067',
 'WP_0081',
 'WP_0060',
 'WP_0134',
 'WP_0074',
 'WP_0133',
 'WP_0014',
 'WP_0028',
 'WP_0001',
 'WP_0094',
 'WP_0015',
 'WP_0089',
 'WP_0108',
 'WP_0103',
 'WP_0041',
 'WP_0055',
 'WP_0030',
 'WP